# Tidy and explore tabular data with pandas
#### A *Python for Reproducible Research* workshop

Welcome! In this lesson, you'll get a crash course in bringing your tabular research data into Python. 

We will focuses on **pandas**, a key part of the scientific Python ecosystem. We'll also introduce you to **NumPy**, the powerful framework underlying pandas and **matplotlib**, a visualization library already integrated into pandas. (We'll use the **seaborn** visualization library briefly too). Everything we do today will happen in this **Jupyter Notebook** - a tool for composing and sharing computational narratives that is becoming increasingly popular across the sciences.

At the end, you'll have written a data analysis recipe you can apply to future datasets and research questions. One way to think of this kind of recipe is a **data science pipeline**.

In [ ]:
from IPython.display import Image

In [ ]:
Image('https://d33wubrfki0l68.cloudfront.net/795c039ba2520455d833b4034befc8cf360a70ba/558a5/diagrams/data-science-explore.png')

*A pipeline for exploratory data analysis (Grolemund and Wickham, 2017)*

Here are the pipeline stages we'll focus on today:

* **Stage 0:** Identify research questions & related data
* **Stage 1:** Import your data into a pandas dataframe
* **Stage 2:** Tidy your data: identify and respond to weirdness!
* **Stage 3:** Explore data: summarize, visualize, plot

And we'll show you what you'll need for one additional step:

* **Stage 4:** Evaluate modeling methods (e.g. classification)

## 0. Identify research questions & related data

### Case study: Breast Cancer Dianostic Dataset

In [ ]:
Image('https://upload.wikimedia.org/wikipedia/commons/8/8b/Breast_fibroadenoma_by_fine_needle_aspiration_%282%29_PAP_stain.jpg', width = 600)

*Fine needle aspiration of fibroadenoma, a type of benign breast tumor. (Source: Wikimedia)*

Today, we'll be using the **Wisconsin Diagnostic Breast Cancer (WDBC) dataset**. 

The [original repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+\(Diagnostic\)) contains several versions of the breast cancer data. For today we're interested in Dr. Wolberg's original encodings of the FNA image data, which uses a ranking of 1 to 10 to describe a variety of attributes such unifority of cell shape and size, mitoses and normal nuclei count, margin adhesion, clump thickness, and so on.

Let's suppose we are researchers at Big University Oncology Institute working with fine needle aspirate (FNA) images of breast masses. Our long-term goal is to increase the efficacy of clinicians in distinguishing malignant from non-malignant masses in FNA digital images.

Given our research questions, how might we wish to explore a dataset where observations correspond to fine needle aspiration (FNA) images of breast growths? What's important to us?

(**Edit this box to include a question you would like to investigate in this data.**)

## 1. Import your data into a pandas dataframe

Let's get started. First, let's import the pandas and numpy library:

In [ ]:
# import pandas and numpy; assign aliases pd and np

In [ ]:
remote_data_URL = 'https://raw.githubusercontent.com/zoews/tidy-tuesday/master/wisconsin_data.csv'

In [ ]:
# read CSV into a dataframe using the appropriate pandas method
# assign this new dataframe to the variable diagnostic_data

In pandas, a DataFrame is a data type that allows us to use special functions. When a function is relative to an object, we call it a **method** and access it with a period at the end of the object. So, for example, print is a standalone function:

```
print("Hello world")
```

But .head() is a pandas **method** that must be called on a pandas DataFrame:

```
diagnostic_data.head()
```

.head() usefully shows us the first several rows in our DataFrame along with row and column labels. Let's try it below, passing in `20` to show the first 20 rows:

In [ ]:
# your code here

Oh no! Take a look at the header line. What do you think is going on here?

In [ ]:
# read csv data into diagnostic_data again, but this time specify no header

Now let's take a look:

In [ ]:
diagnostic_data.head()

Okay, halfway there! We can use the DataFrame as is, but it will be frustrating not to rely on actual column names. We need to dig into the data documentation itself to figure out the best labels in this case. 

Here's the relevant part of our data readme:

```
7. Attribute Information: (class attribute has been moved to last column)

   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

To assign a header to a DataFrame without column labels (or to replace existing Column labels), we can use a list of strings.

In [5]:
attribute_labels = ['sample_code', 
                    'clump_thickness', 
                    'uniformity_cell_shape', 
                    'uniformity_cell_size',
                    'marginal_adhesion',
                    'single_epi_cell_size',
                    'bare_nuclei',
                    'bland_chrom',
                    'norm_nuclei',
                    'mitoses',
                    'mass_class'
                   ]

In [ ]:
# assign the labels to the .columns attribute of our dataframe

In [ ]:
diagnostic_data.head()

You should see a nicely labeled table output to the Notebook above. Congrats! Your data is definitively imported into a DataFrame.

Note that pandas will create an index and assign ordered values beginning at 0 by default. You can override this behavior (as in, you could set sample_code as the index) but it's helpful to have this index in place for future use, e.g. if we wanted to reverse the order of our data and sample_code wasn't sequential. [Here is a helpful resource](https://t.co/swUeOFoc33?amp=1) for learning more about index behavior in pandas.

## 2. Tidy your data: identify and respond to weirdness!

For us to consider a dataset a tidy dataset, it must follow three principles (Wickham, 2013):

> 1. Each variable forms a column.
> 2. Each observation forms a row.
> 3. Each type of observational unit forms a table

We can also express this visually:

In [ ]:
Image('https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png', width=800)

(Source: ["12. Tidy Data"](https://r4ds.had.co.nz/tidy-data.html), *R for Data Science*. Wickham and Grolemund, 2017)

We've already made one significant change to make our DataFrame **tidy** - manually declaring a header! If we leave the distinction between header and first row ambiguous, we lose the integrity of the tidy data structure where each row corresponds to an observation, and *only* an observation.

Another concern is empty, missing, or **null values**. 

### Hunting for nulls

To check for null values in our diagnostic data, we may wish to use the following syntax: `diagnostic_data.isnull().any()`. What this does:

* 'diagnostic_data' is our DataFrame of FNA image data observations
* `.isnull()` is a method that returns True or False if a null value is found, per value in our dataframe
* `.any()` looks for a single True value in any column and returns the outcome.

Let's look at the result:

In [2]:
# check if nulls exists in any observation, by variable

Looks good, right? However, we are assuming that pandas would spit out nulls in the DataFrame - what if our DataFrame is not operating under these principles?

Let's take a look using the `.info()` method (or we could use the `.dtypes` attribute, your choice!)


In [ ]:
# inspect the data types in your dataframe

What do you notice?

The `.unique()` method can be used on a DataFrame column to show all unique values. For instance, if we want to see all of the unique values for mitoses, we could run the following:

In [ ]:
diagnostic_data.mitoses.unique()

Write code below to view the unique values in our DataFrame showing unexpected behavior:

In [ ]:
# Write code below that calls the unique method on the column we are investigating:

# Your code here

To chck how prevelant the unexpected value is in our data, let's replace the `.unique()` method call with `.value_counts()`, which provides counts for each unique value.

In [ ]:
# Write code below that generates counts for each unique value in the column we are investigating.

To treat this variable as numeric, but also include null values, we must use the floating point numeric data type. If we try to convert this data directly to floating point right now, however, it will produce an error. We thus need to use a two-step process:

* `.replace()` to convert our offending character to NumPy NaN object, which you can generate with `np.nan`
* `.astype()` which will allow us to coerce eligible data to floating point numeric, or `'float64'`

In [3]:
# step 1: replace strange characters with np.nan in our target column

In [4]:
# step 2: coerce the entire column into 'float64' data type

Let's check to see if our DataFrame now includes nulls that pandas can recognize, and also stores bare-nuclei in the correct data type:

In [ ]:
# call .isnull().any() in your DataFrame

In [ ]:
# check .info() on the DataFrame

Now you must make a decision: do you exclude observations with null values? Why or why not? If you wanted to exclude those observations, you could filter by null or non-null.

To set a filter condition, you use brackets with a statement to evaluate in Python. Ex:

In [ ]:
diagnostic_data[diagnostic_data.clump_thickness > 5]

To check for a null value, pandas requires us to use the `.isnull()` method which will return either true or false. If we want to negate this, we can use a `!` beforehand, like so:

In [ ]:
diagnostic_data[!diagnostic_data.clump_thickness.isnull()]

How would you use this filter to exclude nulls where we expect to find them?

In [5]:
# Your code below -- assign if you wish, evaluate but don't assign if you prefer!

### Encode benign/malignant class as categorical

In [ ]:
# find counts per class

We just used the `.replace()` method above to properly note nulls. In addition to replacing a single value, we can replace multiple values at the same time. To do this, we can pass in a dictionary.

Uncomment the block below and replace the number-string pairs with the class labels provided for our data (see the '7. Attribute Information' table above)

In [ ]:
# to_replace = {#: '???',
#              #: '???'}

Now, let's coerce the entire column to be of type 'category'. Enter the code below.

In [ ]:
# coerce relevant variable to category type

In [6]:
# check your work!

### Anything else?

In [10]:
# hint: nunique() gives count of unique values (as opposed to total number of values)

## 3. Explore your data: summarize, visualize, transform


REMINDER: What are our research questions?

Let's identify some specific tasks that may be helpful:
* Learn the overall size, shape, and scope of our FNA imaging data.
* Summarize the distribution for each image attribute.
* Explore possible correlations between image attributes.
* Aggregate observations by class and compare

### a. Learn the overall size, shape, and scope of our FNA imaging data.

When getting oriented to a new dataset, I like to see a few examples of observations, and also make sure the data is in a format I expect.

We've already seen `.head()` and `.info()`. Let's also add in `.tail()`, which just shows the bottom 5 items. We can also pass in a parameter to see the nth rows from the bottom.

In [ ]:
# Your code

If you want to quickly view the overall dimensions of your DataFrame, the most concise way is to use `.shape`

Unlike .head(), which is a **method** (or function that belongs to our DataFrame object), .shape is an **atribute** meaning that it returns some data about the object. 

In [ ]:
# Do you think shape uses .shape or .shape() syntax? Try it out

### b. Summary statistics and distributions for each image attribute.

We now have a big-picture view of how our dataframe is organized, but we don't yet know much about the individual columns. What is the distribution of values within a given column? How do they compare to one another?

First, run the `.columns` attribute call on our dataframe to remind us of all available columns:

In [12]:
# replace with your code

Now, let's get a visual sense of the distirbution of values within each column. First, let's see this as a big matrix using the .describe() method.

In [8]:
# describe your dataframe

Pandas supports visualization using the matplotlib library natively, which is very helpful for our purposes. To make this work within a notebook, we simply need to import matplotlib, and then specify that we want to see inline visualizations with a "magic command" (the magic command is specific to Jupyter Notebooks):

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt

We can generate side-by-side histograms of all our columns by calling the `.hist()` method on our DataFrame. It's not necessary to specify paramters, but for readability, it's helpful to set the `figsize` parameter to an `(x, y)` tuple, in inches:

In [14]:
# our_plot = diagnostic_data.hist(figsize=(12,12))

We can pair this code with the column select syntax `DataFrame[["Column A", "Column B"...]]` to compare just a subset of columns. Unlike the dot format of referencing a column (which is my favorite for most situations), the bracket format allows you to pass in a list of columns, which is helpful for subsetting. If you write code this way, note that you need two brackets, and the column names must now be passed in as strings (and thus must be surrounded by quotation marks).


In [15]:
# generate a comparison_plot that only focuses on two columns, in contrast to the our_plot code above

In [24]:
## Add your own code to compare the distribution of two other variables here!

Now let's explore a given column. You can always start with describe:

In [25]:
# call describe on clump_thickness

But you may wish to extract specific information such as the mean, median, sum, count, etc. All of these can be called as methods on pandas DataFrame column (which is equivalent to a pandas Series), such as `df.MY_COLUMN.sum()` or `df.MY_COLUMN.mean()`. Try it out below:

In [1]:
# your code here

You can also write this method call using the bracket notation for referencing a column, such as df['MY_COLUMN'].sum()

In [2]:
# your code here

### c. Possible correlations between image attributes.

Let's visualize correlations!

We can start with calling the .corr() method on our DataFrame

In [18]:
# generate correlation matrix

In [19]:
# now, assign correlation matrix to variable
# use plt.matshow() to generate a matrix visualization
# then use plt.show() to print it out to the Notebook

matplotlib is only so-so at visualization correlations. Let's use seaborn instead.

In [21]:
import seaborn as sns

In [22]:
# call sns.heatmap()
# this variable takes three parameters: the correlation matrix
# xticklabels, which we can assign to the .columns attribute of our correlation matrix
# and yticklabels, which we can ALSO assign to the .columns attribute of our correlation matrix

If you'd like to try more seaborn visualizations, take a look at some [more examples](https://jakevdp.github.io/PythonDataScienceHandbook/04.14-visualization-with-seaborn.html)

### d. Aggregate observations by class and compare

Here we will focus on the `.groupby()` method and the concept of the **index** in a DataFrame

In [3]:
# group by our output class benign/malignant and describe

In [ ]:
# group by our output class malignant and call hist

`.hist()` is a good start, but let's use seaborn to see a little more clearly.

Let's use [catplot](https://seaborn.pydata.org/tutorial/categorical.html)

## 4. Evaluate modeling methods (e.g. classification)

We've tidied up our dataset, including dealing with headers and nulls; glimpsed the design, shape, and typical observations within our datset; explored summary statistics and distributions of values within columns; and evaluated covariance between columns with correlation plots and heat maps. 

Each of these tasks are valuable in their own right, no matter what analytical method we wish to pursue next. 

### Let's save our work.

In [4]:
# diagnostic_data.to_csv("wisconsin_data_clean.csv")

### Avenues for subsequent analysis (e.g. designing a machine learning classification task)

The data work you've done in pandas is *particularly* relevant if we plan to approach this research question

> we are interested in whether patterns exist in imaging data that would improve the ability of clinicians to correctly dinstinguish malignant and non-malignant tumors

as the classification task

> Given the output class of "malignant" or "benign" in our observations, can we generate an accurate and reliable prediction from observations about Fine Needle Aspiration images (also contained within our observations)? If so, what variables are singificant in predicting an outcome class effectively? Among those important predictors, how would a change in measurement value influence the likely outcome, and how confident are we in that influence?

Now that we have our data in a tidy dataframe, those observations include a outcome class variable as well as several storng candidate for predictor variables. This is an excellent task to pursue via the supervised machine learning method of [binary classification](https://www.sciencedirect.com/topics/computer-science/binary-classification).

In the Python ecosystem, most if not all popular machine learning packages will work natively with pandas DataFrames. Look out for a future lesson using random forest classifers with scikit-learn on this very dataset you have prepared!

## Works cited & further reading

## Appendix: Full readme file

Full readme file accompanying data:

```
Citation Request:
   This breast cancer databases was obtained from the University of Wisconsin
   Hospitals, Madison from Dr. William H. Wolberg.  If you publish results
   when using this database, then please include this information in your
   acknowledgements.  Also, please cite one or more of:

   1. O. L. Mangasarian and W. H. Wolberg: "Cancer diagnosis via linear 
      programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

   2. William H. Wolberg and O.L. Mangasarian: "Multisurface method of 
      pattern separation for medical diagnosis applied to breast cytology", 
      Proceedings of the National Academy of Sciences, U.S.A., Volume 87, 
      December 1990, pp 9193-9196.

   3. O. L. Mangasarian, R. Setiono, and W.H. Wolberg: "Pattern recognition 
      via linear programming: Theory and application to medical diagnosis", 
      in: "Large-scale numerical optimization", Thomas F. Coleman and Yuying
      Li, editors, SIAM Publications, Philadelphia 1990, pp 22-30.

   4. K. P. Bennett & O. L. Mangasarian: "Robust linear programming 
      discrimination of two linearly inseparable sets", Optimization Methods
      and Software 1, 1992, 23-34 (Gordon & Breach Science Publishers).

1. Title: Wisconsin Breast Cancer Database (January 8, 1991)

2. Sources:
   -- Dr. WIlliam H. Wolberg (physician)
      University of Wisconsin Hospitals
      Madison, Wisconsin
      USA
   -- Donor: Olvi Mangasarian (mangasarian@cs.wisc.edu)
      Received by David W. Aha (aha@cs.jhu.edu)
   -- Date: 15 July 1992

3. Past Usage:

   Attributes 2 through 10 have been used to represent instances.
   Each instance has one of 2 possible classes: benign or malignant.

   1. Wolberg,~W.~H., \& Mangasarian,~O.~L. (1990). Multisurface method of 
      pattern separation for medical diagnosis applied to breast cytology. In
      {\it Proceedings of the National Academy of Sciences}, {\it 87},
      9193--9196.
      -- Size of data set: only 369 instances (at that point in time)
      -- Collected classification results: 1 trial only
      -- Two pairs of parallel hyperplanes were found to be consistent with
         50% of the data
         -- Accuracy on remaining 50% of dataset: 93.5%
      -- Three pairs of parallel hyperplanes were found to be consistent with
         67% of data
         -- Accuracy on remaining 33% of dataset: 95.9%

   2. Zhang,~J. (1992). Selecting typical instances in instance-based
      learning.  In {\it Proceedings of the Ninth International Machine
      Learning Conference} (pp. 470--479).  Aberdeen, Scotland: Morgan
      Kaufmann.
      -- Size of data set: only 369 instances (at that point in time)
      -- Applied 4 instance-based learning algorithms 
      -- Collected classification results averaged over 10 trials
      -- Best accuracy result: 
         -- 1-nearest neighbor: 93.7%
         -- trained on 200 instances, tested on the other 169
      -- Also of interest:
         -- Using only typical instances: 92.2% (storing only 23.1 instances)
         -- trained on 200 instances, tested on the other 169

4. Relevant Information:

   Samples arrive periodically as Dr. Wolberg reports his clinical cases.
   The database therefore reflects this chronological grouping of the data.
   This grouping information appears immediately below, having been removed
   from the data itself:

     Group 1: 367 instances (January 1989)
     Group 2:  70 instances (October 1989)
     Group 3:  31 instances (February 1990)
     Group 4:  17 instances (April 1990)
     Group 5:  48 instances (August 1990)
     Group 6:  49 instances (Updated January 1991)
     Group 7:  31 instances (June 1991)
     Group 8:  86 instances (November 1991)
     -----------------------------------------
     Total:   699 points (as of the donated datbase on 15 July 1992)

   Note that the results summarized above in Past Usage refer to a dataset
   of size 369, while Group 1 has only 367 instances.  This is because it
   originally contained 369 instances; 2 were removed.  The following
   statements summarizes changes to the original Group 1's set of data:

   #####  Group 1 : 367 points: 200B 167M (January 1989)
   #####  Revised Jan 10, 1991: Replaced zero bare nuclei in 1080185 & 1187805
   #####  Revised Nov 22,1991: Removed 765878,4,5,9,7,10,10,10,3,8,1 no record
   #####                  : Removed 484201,2,7,8,8,4,3,10,3,4,1 zero epithelial
   #####                  : Changed 0 to 1 in field 6 of sample 1219406
   #####                  : Changed 0 to 1 in field 8 of following sample:
   #####                  : 1182404,2,3,1,1,1,2,0,1,1,1

5. Number of Instances: 699 (as of 15 July 1992)

6. Number of Attributes: 10 plus the class attribute

7. Attribute Information: (class attribute has been moved to last column)

   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)

8. Missing attribute values: 16

   There are 16 instances in Groups 1 to 6 that contain a single missing 
   (i.e., unavailable) attribute value, now denoted by "?".  

9. Class distribution:
 
   Benign: 458 (65.5%)
   Malignant: 241 (34.5%)

```